In [1]:
!pip install jsonlines
!pip install transformers

In [2]:
import jsonlines
import json
import torch
from transformers import AutoModel, AutoTokenizer

In [3]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open('drive/MyDrive/NeuralDB/data/v2.4_25/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

train_dbs, test_dbs, dev_dbs = [], [], []

for json_str in json_list:
    result = json.loads(json_str)
    train_dbs.append(result)
    # print(f"result: {result}")
    # print(isinstance(result, dict))

with open('drive/MyDrive/NeuralDB/data/v2.4_25/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    test_dbs.append(result)


with open('drive/MyDrive/NeuralDB/data/v2.4_25/dev.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    dev_dbs.append(result)

In [ ]:
with jsonlines.open('data/NDB Data/v2.4_25/train.jsonl') as f:
    result = [json.loads(jline) for jline in f.splitlines()]
    counter = 1
#     for line in f.iter():
#         print(len(line['facts']), len(line['queries']), len(line['metadata']))
#         counter += 1
#         # for fact in line['facts']:
#         #     print(fact)
#         # break
    
# print('Databases : ', counter)

In [ ]:
import jsonlines

with jsonlines.open('data/NDB Data/v2.4_25/train.jsonl') as f:

    for line in f.iter():
        print(line['metadata']['raw'])
        # for item in line['metadata']['raw']:
        #     print(item.keys())
        break

In [ ]:
import jsonlines

with jsonlines.open('data/NDB Data/v2.4_25/train.jsonl') as f:

    for line in f.iter():
        print(line['metadata']['raw'][0])
        break
        # for item in line['metadata']['raw']:
        #     print(item['valid_hypotheses'])
        #     break

In [ ]:
import jsonlines

with jsonlines.open('data/NDB Data/v2.4_25/train.jsonl') as f:

    for line in f.iter():
        # print(line['facts'])
        for fact in line['queries']:
            print(fact)
        break

In [5]:
MODEL_NAME = "t5-small"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
demo_db = train_dbs[0]

query = demo_db['queries'][1]['query']
query_facts = demo_db['queries'][1]['facts']

query_facts = [demo_db['metadata']['raw'][fact[0]]['parse_targets'] for fact in query_facts]
query_facts = [fact for sub_fact in query_facts for fact in sub_fact ]

query_facts
text = ''
for fact in query_facts:
    text += fact + ' '

text = text[:-1]

In [ ]:
encoding = tokenizer(query, text)

In [ ]:
encoding

In [ ]:
query_facts

In [ ]:
for x in encoding["input_ids"]:
    print(tokenizer.decode(x))

In [7]:
train_dataset = []
for db in train_dbs:
  for query in db['queries']:
    query_text = query['query']
    query_facts = query['facts']

    query_facts = [db['metadata']['raw'][fact[0]]['parse_targets'] for fact in query_facts]
    query_facts = [fact for sub_fact in query_facts for fact in sub_fact ]

    text = ''

    for fact in query_facts:
      text += fact + ' '

    text = text[:-1]
    train_dataset.append([query_text, text])

In [8]:
test_dataset = []
for db in test_dbs:
  for query in db['queries']:
    query_text = query['query']
    query_facts = query['facts']

    query_facts = [db['metadata']['raw'][fact[0]]['parse_targets'] for fact in query_facts]
    query_facts = [fact for sub_fact in query_facts for fact in sub_fact ]

    text = ''

    for fact in query_facts:
      text += fact + ' '

    text = text[:-1]
    test_dataset.append([query_text, text])


In [9]:
dev_dataset = []
for db in dev_dbs:
  for query in db['queries']:
    query_text = query['query']
    query_facts = query['facts']

    query_facts = [db['metadata']['raw'][fact[0]]['parse_targets'] for fact in query_facts]
    query_facts = [fact for sub_fact in query_facts for fact in sub_fact ]

    text = ''

    for fact in query_facts:
      text += fact + ' '

    text = text[:-1]
    dev_dataset.append([query_text, text])

In [10]:
!pip install datasets

In [11]:
import pandas as pd
from datasets import Dataset, DatasetDict

train_df = pd.DataFrame(train_dataset, columns = ['query', 'facts'])
test_df = pd.DataFrame(test_dataset, columns = ['query', 'facts'])
dev_df = pd.DataFrame(dev_dataset, columns = ['query', 'facts'])

train_d = Dataset.from_pandas(train_df)
test_d = Dataset.from_pandas(test_df)
dev_d = Dataset.from_pandas(dev_df)

dataset_dict = {"train": train_d,
           "test": test_d,
           "unsupervised": dev_d}

d = DatasetDict(dataset_dict)
d

DatasetDict({
    train: Dataset({
        features: ['query', 'facts'],
        num_rows: 33474
    })
    test: Dataset({
        features: ['query', 'facts'],
        num_rows: 5272
    })
    unsupervised: Dataset({
        features: ['query', 'facts'],
        num_rows: 5252
    })
})

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["query"],examples["facts"], padding="max_length", truncation=True)
tokenized_datasets = d.map(tokenize_function, batched=True)

In [15]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=2,
    weight_decay=0.01,
    output_dir="content/",

)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets.get("train"),
    eval_dataset=tokenized_datasets.get("test"),
)





PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5Model.forward` and have been ignored: query, facts.
***** Running training *****
  Num examples = 33474
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 10044


ValueError: ignored

In [19]:
torch.cuda.memory_stats()

OrderedDict([('active.all.allocated', 603),
             ('active.all.current', 267),
             ('active.all.freed', 336),
             ('active.all.peak', 407),
             ('active.large_pool.allocated', 407),
             ('active.large_pool.current', 145),
             ('active.large_pool.freed', 262),
             ('active.large_pool.peak', 268),
             ('active.small_pool.allocated', 196),
             ('active.small_pool.current', 122),
             ('active.small_pool.freed', 74),
             ('active.small_pool.peak', 141),
             ('active_bytes.all.allocated', 8056851456),
             ('active_bytes.all.current', 3144309248),
             ('active_bytes.all.freed', 4912542208),
             ('active_bytes.all.peak', 6151470080),
             ('active_bytes.large_pool.allocated', 7979401216),
             ('active_bytes.large_pool.current', 3068395520),
             ('active_bytes.large_pool.freed', 4911005696),
             ('active_bytes.large_pool.peak', 6